In [1]:
import pandas as pd
from yaml import load
from yaml import CLoader as Loader
from planktonsdm.tune import tune
from planktonsdm.functions import example_data
from sklearn.preprocessing import OneHotEncoder

In [2]:
#If you want to debug the package without repackaging, you can read files locally:

import sys
sys.path.insert(0, '/home/phyto/planktonSDM/planktonsdm/')
from tune import tune
from functions import example_data

In [3]:
# Setting up a regressor model

with open('/home/phyto/planktonSDM/configuration/ensemble_regressor.yml', 'r') as f:
    model_config = load(f, Loader=Loader)

predictors = model_config['predictors']
d = pd.read_csv("/home/phyto/planktonSDM/data/abundances_environment.csv")
species = 'Emiliania huxleyi'
d.dropna(subset=species, inplace=True)
d.dropna(subset='FID', inplace=True)
X_train = d[predictors]
y = d[species]
X_predict = pd.read_csv("/home/phyto/planktonSDM/data/envdata_april_surface.csv")
X_predict.set_index(["time", "depth", "lat", "lon"], inplace=True)
X_predict = X_predict[X_train.columns]


m = tune(X_train, y, model_config, regions="FID")

upsampling = True


In [4]:
'''
1-phase Random forest regressor
'''
m.train(model="rf", regressor=True)

training regressor
Fitting 10 folds for each of 4 candidates, totalling 40 fits
exported model to: /home/phyto/CoccoML/ModelOutput/rf/model/Emiliania huxleyi_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/rf/scoring/Emiliania huxleyi_reg.sav
reg rRMSE: 356%
reg rMAE: 97%
reg R2: 0.1
execution time: 7.152557373046875e-07 seconds


In [5]:
'''
1-phase XGboost regressor
'''
m.train(model="xgb", regressor=True)

training regressor
Fitting 10 folds for each of 72 candidates, totalling 720 fits
exported model to: /home/phyto/CoccoML/ModelOutput/xgb/model/Emiliania huxleyi_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/xgb/scoring/Emiliania huxleyi_reg.sav
reg rRMSE: 358%
reg rMAE: 86%
reg R2: 0.14
execution time: 7.152557373046875e-07 seconds


In [6]:
'''
1-phase KNN regressor
'''
m.train(model="knn", regressor=True)

training regressor
Fitting 10 folds for each of 1 candidates, totalling 10 fits


exported model to: /home/phyto/CoccoML/ModelOutput/knn/model/Emiliania huxleyi_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/knn/scoring/Emiliania huxleyi_reg.sav
reg rRMSE: 367%
reg rMAE: 97%
reg R2: 0.0
execution time: 9.5367431640625e-07 seconds
